In [ ]:
from cosapp.systems import System

from pythonocc_helpers.create import CreateCircle, CreateExtrusion, CreatePlane, CreateLine, CreateRevolution, CreateAxis, CreateWire, CreateTopology
from pythonocc_helpers.transform import Sweep, Translate, Scale
from pythonocc_helpers.render import JupyterThreeJSRenderer

from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeFace


def face_from_wires(plane, *wires):
    builder = BRepBuilderAPI_MakeFace(plane)  
    for w in wires:
        builder.Add(CreateWire.from_element(w))
        
    return builder.Shape()


class PipeGeometry(System):

    def setup(self):
        self.add_inward("length", 1., unit="m", desc="Pipe length")
        self.add_inward("diameter", 0.1, unit="m", desc="Pipe diameter")
        self.add_inward("thickness", 1e-3, unit="m", desc="Pipe thickness")

        self.add_outward("shape", None, desc="Shape")

    def compute(self):
        inner_radius = self.diameter / 2.
        c1 = CreateCircle.from_radius_and_center(inner_radius, (0., 0., 0.), (1., 0., 0.))
        c2 = CreateCircle.from_radius_and_center(inner_radius - self.thickness, (0., 0., 0.), (1., 0., 0.))
        s = face_from_wires(CreatePlane.yoz(), c1, c2.Reversed())
        self.shape = CreateExtrusion.surface(s, (self.length, 0., 0.))

        
class HeatExchangerGeometry(System):

    def setup(self):
        
        self.add_child(PipeGeometry("pipe"), pulling=["length", "diameter", "thickness"])

        self.add_inward("branch_count", 1, unit="", desc="Branch count")
        self.add_inward("branch_spacing", 0.1, unit="m", desc="Distance between branches")
        self.add_inward("depth", 2.5, unit="m", desc="Depth")

        self.add_outward("shape", None, desc="Shape")

    def compute(self):
        assert self.branch_spacing >= self.diameter

        pipe_shape = self.pipe.shape
        print(self.branch_count / 2)
        pipes = [Translate.from_vector(pipe_shape, (0., -self.depth, self.branch_spacing * (i - (self.branch_count - 1) / 2)), inplace=False) for i in range(self.branch_count)]

        self.shape = Scale.from_factor(CreateTopology.make_compound(*pipes), 0.1, inplace=False)


class PipeMaterial(System):

    def setup(self):

        self.add_inward("lambda", unit="J/K/m**2", desc="Pipe diameter")
        self.add_inward("density", unit="kg/m**3", desc="Pipe density")


class HeatExchanger(System):

    def setup(self):

        self.add_child(PipeMaterial("material"))
        self.add_child(HeatExchangerGeometry("geometry"))

In [ ]:
exchanger = HeatExchanger("exchanger")

In [ ]:
render = JupyterThreeJSRenderer(view_size=(1400, 800), camera_target=(1., 1., 0.), camera_position=(1., 1., -2.))

render_row = render.add_shape(exchanger.geometry.shape, uid="exchanger", face_color="#156289", opacity=1.)
render_row.linear_deflection = 0.15
render_row.angular_deflection = 0.15
render.show()

In [ ]:
exchanger.geometry.diameter = 0.4
exchanger.geometry.length = 20.
exchanger.geometry.thickness = 0.004
exchanger.geometry.branch_count = 10
exchanger.geometry.branch_spacing = 1.2

exchanger.run_drivers()

render.update_shape(exchanger.geometry.shape, uid="exchanger");